In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

데이터 불러오기

In [ ]:
air_res = pd.read_csv('../input/recruit-restaurant-visitor-forecasting/air_reserve.csv.zip')
air_store = pd.read_csv('../input/recruit-restaurant-visitor-forecasting/air_store_info.csv.zip')
air_visit = pd.read_csv('../input/recruit-restaurant-visitor-forecasting/air_visit_data.csv.zip')
date = pd.read_csv('../input/recruit-restaurant-visitor-forecasting/date_info.csv.zip')
hpg_res = pd.read_csv('../input/recruit-restaurant-visitor-forecasting/hpg_reserve.csv.zip')
hpg_store = pd.read_csv('../input/recruit-restaurant-visitor-forecasting/hpg_store_info.csv.zip')
id_rel = pd.read_csv('../input/recruit-restaurant-visitor-forecasting/store_id_relation.csv.zip')

In [ ]:
air_res.head()

In [ ]:
air_store.head()

In [ ]:
air_visit.head()

In [ ]:
date.head()

In [ ]:
id_rel.head()

In [ ]:
air_res.air_store_id.nunique() # air 시스템으로 예 약 가능한 레스토랑 수

In [ ]:
hpg_res.hpg_store_id.nunique() # hpg 시스템으로 예약 가능한 레스토랑 수

In [ ]:
id_rel.air_store_id.nunique() # air 시스템과 hpg 시스템으 모두 예약 가능한 레스토랑 수

In [ ]:
# 테이블을 병합하기 전에 이름을 바꾼다
# air_store.rename(columns={"air_genre_name": "genre_name", " B": "c"})

In [ ]:
# 테이블 병합
air = pd.merge(air_res, air_store, on='air_store_id')
hpg = pd.merge(hpg_res, hpg_store, on='hpg_store_id')

In [ ]:
air_rel = pd.merge(air, id_rel, how='left', on='air_store_id')
hpg_rel = pd.merge(hpg, id_rel, how='left', on='hpg_store_id')
full = pd.merge(air_rel, hpg_rel, how='outer')

In [ ]:
print("In air reservations are: %d \nIn hpg reservations are: %d \nIn both systems at once there are: %d" % \
      (air.shape[0], hpg.shape[0], id_rel.shape[0]))
print("So, totally must be: %d reservations." \
      % (full.shape[0]))

In [ ]:
# 'visit_datetime'열과 reserve_datetime'열을 object type에서 data/time type으로 바꾼다
full['visit_datetime'] = pd.to_datetime(full['visit_datetime'])
full['reserve_datetime'] = pd.to_datetime(full['reserve_datetime'])

In [ ]:
full.info()

In [ ]:
full.head()

In [ ]:
full['air_store_id'] = full['air_store_id'].fillna('0')
full['hpg_store_id'] = full['hpg_store_id'].fillna('0')
full['air_genre_name'] = full['air_genre_name'].fillna('0')
full['hpg_genre_name'] = full['hpg_genre_name'].fillna('0')


In [ ]:
full.loc[(full['air_genre_name'] != '0'), 'store_id'] = full['air_store_id']
full.loc[(full['air_genre_name'] == '0'), 'store_id'] = full['hpg_store_id']

# Create column 'store_genre_name', where all genres of restaurants will be collected together
full.loc[(full['air_genre_name'] != '0'), 'store_genre_name'] = full['air_genre_name']
full.loc[(full['air_genre_name'] == '0'), 'store_genre_name'] = full['hpg_genre_name']

# Create column 'air_hpg_link', to save connection between restaurants that are in both sources
full.loc[(full['air_store_id'] != '0') & (full['hpg_genre_name'] != '0'), 'air_hpg_link'] = full['air_store_id']
full.loc[(full['air_store_id'] != '0') & (full['hpg_genre_name'] == '0'), 'air_hpg_link'] = full['hpg_store_id']

# Create column 'store_genre_name', where all genres of restaurants will be collected together
full.loc[(full['air_genre_name'] != '0'), 'area_name'] = full['air_area_name']
full.loc[(full['air_genre_name'] == '0'), 'area_name'] = full['hpg_area_name']

full['air_hpg_link'] = full['air_hpg_link'].fillna('0')


In [ ]:
full.head()